### Word Group In Youtube

In [3]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [4]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [5]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 1
    word_usage_max = 30
elif word_end == 20000:
    word_usage_min = 1
    word_usage_max = 40
else:
    word_usage_min = 1
    word_usage_max = 50

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 25  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45

# start end time result shift
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:1000
file_ext:6
unique_word_ratio:80
include_wordgroup_ratio:80
word_usage_min:1
word_usage_max:30


In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/3-Word Group In Youtube"

Path(path).mkdir(parents=True, exist_ok=True)

In [7]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_adjust_usage_dict):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        #count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any(True if word_adjust_usage_dict[f"{word}"] < dict_list_count[f"{word}"] else False for word in words) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [8]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []
    # check words in word_list
    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass
    # add index plus 1 and compare with current and next index (Is it continuous or not?)
    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    # index convert to word
    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [9]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [10]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [11]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [12]:
def word_max_usage_limit(df, word_column_name, word_freq_column_name, min_word_usage):
    '''
    word_max_usage_limit(df, word_column_name, word_freq_column_name, min_word_usage)
    '''
    word_adjust_usage_dict = {}
    min_frequency = min(df[f"{word_freq_column_name}"])
    for word,frequency in zip(df[f"{word_column_name}"], df[f"{word_freq_column_name}"]):        
        word_usage_var = int((frequency*min_word_usage)/min_frequency)
        word_adjust_usage_dict[f"{word}"] = word_usage_var
    return word_adjust_usage_dict   

In [13]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Select Word

In [14]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all  # word checked with picture word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987890,karneleme,5
987891,karnaya,5
987892,dörtlümüzün,5
987893,karnavalınız,5


In [15]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [16]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,resmi,68287
996,veriyor,68163
997,okul,68160
998,suçlu,68124


In [17]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [18]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [19]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

1000

##### Check Word From Picture Word

In [20]:
image_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/\
Data/1000 Words/Genome_Picture_In_Picture_Select_Rename"

In [21]:
file_list = os.listdir(f"{image_path_folder}")
#file_list

In [22]:
image_file_name = []
for file in file_list:
    # file and extention
    file_without_ext = os.path.splitext(file)[0]
    image_file_name.append(file_without_ext)

In [23]:
word_set = set(word_list)
image_file_name_set = set(image_file_name)

In [24]:
word_set.difference(image_file_name_set)

set()

In [25]:
image_file_name_set.difference(word_set)

set()

#### Youtube Sentence Analysis

In [26]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [27]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [28]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,3,10.267,11.394,mumu,fI1BI4d5KFU
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [29]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [30]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result.drop_duplicates(subset="sentence", inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, lm, betimlemesi, fi, dizinin","tarafından, bu"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.0,"yaptırılmıştır, derneğine, betimleme, sesli",
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.0,wwwsebederorg,
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.0,mumu,
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"safhada, katsayım, hat, sinir","şu, an, bak"
...,...,...,...,...,...,...,...,...
3934201,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, ediyorlar, ihtiyaç, iyilik...","tercih, ve, güzel, bir, çok, bu"
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"bitirmeden, danimarka, arazilerin, dümdüz, eng...","isterim, şunu, söylemek, da, bir, ki, olduğu"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yarısına, okuluna, yakını, ulaşımd...","gidiyor, insanların, bu"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.3,"alakalı, söyleyeceklerim, şimdilik, danimarkayla","kadar, bu"


In [31]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, lm, betimlemesi, fi, dizinin","tarafından, bu"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.0,"yaptırılmıştır, derneğine, betimleme, sesli",NaN
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.0,wwwsebederorg,NaN
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.0,mumu,NaN
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"safhada, katsayım, hat, sinir","şu, an, bak"
...,...,...,...,...,...,...,...,...
3934201,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, ediyorlar, ihtiyaç, iyilik...","tercih, ve, güzel, bir, çok, bu"
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"bitirmeden, danimarka, arazilerin, dümdüz, eng...","isterim, şunu, söylemek, da, bir, ki, olduğu"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yarısına, okuluna, yakını, ulaşımd...","gidiyor, insanların, bu"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.3,"alakalı, söyleyeceklerim, şimdilik, danimarkayla","kadar, bu"


In [32]:
#df_sentence_ratio_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_All.csv", index=False)

In [33]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, lm, betimlemesi, fi, dizinin","tarafından, bu"
1,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"safhada, katsayım, hat, sinir","şu, an, bak"
2,5,14.213,17.219,kelimelerini lütfen seçerek konuş lütfen,fI1BI4d5KFU,40.0,"seçerek, kelimelerini","konuş, lütfen"
3,6,18.687,19.854,otursana bir tanem,fI1BI4d5KFU,33.3,"otursana, tanem",bir
4,7,19.967,22.805,hah oturtmaya çalıştığına göre demek doğru,fI1BI4d5KFU,50.0,"oturtmaya, hah, çalıştığına","göre, doğru, demek"
...,...,...,...,...,...,...,...,...
2675734,3934200,590.325,594.700,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0,40.0,"zira, insanlarımız, satmak, kıyafetleri, işine...","bizim, ikinci, yerine, el"
2675735,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, ediyorlar, ihtiyaç, iyilik...","tercih, ve, güzel, bir, çok, bu"
2675736,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"bitirmeden, danimarka, arazilerin, dümdüz, eng...","isterim, şunu, söylemek, da, bir, ki, olduğu"
2675737,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yarısına, okuluna, yakını, ulaşımd...","gidiyor, insanların, bu"


In [34]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio = df_adjust_sentence_ratio.drop_duplicates(subset="search_string")
df_adjust_sentence_ratio

<ipython-input-34-f585961066c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
20,25,81.538,84.046,demek o yüzden öyle garip davrandın,fI1BI4d5KFU,83.3,davrandın,"garip, demek, o, öyle, yüzden",demek o yüzden öyle garip
32,38,135.778,139.810,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU,70.0,rezil,"etti, beni, o, deli, herkesin, kadın, içinde",o deli kadın herkesin içinde beni
37,43,147.019,149.009,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU,71.4,karşımdan,"böyle, ama, yapma, git, dedim",ama yapma böyle git dedim git
60,79,302.034,303.874,bana o kadar iyi geldi ki,fI1BI4d5KFU,100.0,NaN,"o, geldi, iyi, ki, kadar, bana",bana o kadar iyi geldi ki
71,93,336.721,339.860,allahım ben tamamen inanamıyorum şu an ya,fI1BI4d5KFU,85.7,allahım,"şu, an, inanamıyorum, ya, ben, tamamen",ben tamamen inanamıyorum şu an ya
...,...,...,...,...,...,...,...,...,...
2675684,3934138,187.205,192.549,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0,66.7,"sevinir, geliyorsunuz, köyleri, görünce","dışarıda, o, sakin, hatta, hale, ki, kadar, insan",o kadar sakin ki dışarıda insan
2675731,3934197,578.076,581.389,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0,66.7,"kıyafet, sektörü","ikinci, var, yani, ciddi, orada, el",yani orada ciddi ciddi ikinci el
2675733,3934199,583.933,590.183,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0,60.0,"kıyafet, hatırlamıyorum, gördüğümü, dükkanı, y...","ikinci, iş, bence, fakat, varsa, el, eğer, da,...",eğer varsa da bence çok iş
2675735,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, ediyorlar, ihtiyaç, iyilik...","tercih, ve, güzel, bir, çok, bu",ve bu çok güzel bir


In [ ]:
## will test
#df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].apply(lambda x: exract_word_group(x, word_list))
#df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
##df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
#df_adjust_sentence_ratio = df_adjust_sentence_ratio.drop_duplicates(subset="search_string")
#df_adjust_sentence_ratio

In [31]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.538000,83.401086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.778000,138.062800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147.019000,148.549769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302.034000,303.874000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,ben tamamen inanamıyorum şu an ya,337.256927,339.860000,allahım ben tamamen inanamıyorum şu an ya,fI1BI4d5KFU
...,...,...,...,...,...
248025,o kadar sakin ki dışarıda insan,188.084392,190.316696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
248026,yani orada ciddi ciddi ikinci el,578.076000,580.178481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
248027,eğer varsa da bence çok iş,587.861571,589.528238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
248028,ve bu çok güzel bir,597.441716,598.724012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [32]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.238000,83.701086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.478000,138.362800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,146.719000,148.849769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,301.734000,304.174000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,ben tamamen inanamıyorum şu an ya,336.956927,340.160000,allahım ben tamamen inanamıyorum şu an ya,fI1BI4d5KFU
...,...,...,...,...,...
248025,o kadar sakin ki dışarıda insan,187.784392,190.616696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
248026,yani orada ciddi ciddi ikinci el,577.776000,580.478481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
248027,eğer varsa da bence çok iş,587.561571,589.828238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
248028,ve bu çok güzel bir,597.141716,599.024012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [33]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81,84,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135,138,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147,149,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302,304,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,ben tamamen inanamıyorum şu an ya,337,340,allahım ben tamamen inanamıyorum şu an ya,fI1BI4d5KFU
...,...,...,...,...,...
248025,o kadar sakin ki dışarıda insan,188,191,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
248026,yani orada ciddi ciddi ikinci el,578,580,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
248027,eğer varsa da bence çok iş,588,590,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
248028,ve bu çok güzel bir,597,599,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [34]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
...,...,...,...,...,...
248025,da o o da ki,61,63,bu bacağı ben uçmak istiyorum demesiydi bunun ...,agGOHteE2GI
248026,o mu o şey o,2237,2240,ha o mu o şey o,99Iaexxewq4
248027,o o o şey o,1183,1186,o o o şey o,_FsOnRDIJQQ
248028,o o o o o o,697,701,olur biz saçında bir dur sevdim be o o o o o o,wXAnHYWPnfw


In [35]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,93
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,77
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,100
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,89
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,94
...,...,...,...,...,...,...
248025,da o o da ki,61,63,bu bacağı ben uçmak istiyorum demesiydi bunun ...,agGOHteE2GI,60
248026,o mu o şey o,2237,2240,ha o mu o şey o,99Iaexxewq4,60
248027,o o o şey o,1183,1186,o o o şey o,_FsOnRDIJQQ,40
248028,o o o o o o,697,701,olur biz saçında bir dur sevdim be o o o o o o,wXAnHYWPnfw,16


In [36]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-36-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-36-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
218262,yok ki ya o da,7361,7363,i lyas abinin haberi yok ki ya o da sevinecek,XREYOwgg6Uk
218263,ya o da mı yok,7065,7067,abi hey barmen gürkan ya o da mı yok,r1eZ_SdK_24
218264,da var şu an o,366,376,çeksem çekmesem mi diye düşünüyorum belki oyna...,EafHmnHMjvg
218265,ki tek şu an o,1381,1383,yeni nesil yazılımlar sistemler deniliyor bura...,Sa8nJcrmBp0


In [37]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-37-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,100
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,92
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,100
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,100
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,100
...,...,...,...,...,...,...
218262,yok ki ya o da,7361,7363,i lyas abinin haberi yok ki ya o da sevinecek,XREYOwgg6Uk,31
218263,ya o da mı yok,7065,7067,abi hey barmen gürkan ya o da mı yok,r1eZ_SdK_24,38
218264,da var şu an o,366,376,çeksem çekmesem mi diye düşünüyorum belki oyna...,EafHmnHMjvg,16
218265,ki tek şu an o,1381,1383,yeni nesil yazılımlar sistemler deniliyor bura...,Sa8nJcrmBp0,12


In [39]:
df_word_group_time_loc_in_sent_result_repeat_ratio.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Not_Word_Usage.csv", index=False)

In [40]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-40-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
48786,o bu ve her gün,893,897,o bu ve her gün,IiptWs5QgDA
48787,ne o boş ver ya,4366,4369,ne o boş ver ya,BeCS4X5oyXQ
48788,bu ay mı yok mu,199,263,bu ay mı yok mu,d4hmTrjuAk0
48789,ya bir de o var,1286,1288,ya bir de o var,Wzuoaygu2dQ


In [41]:
word_adjust_usage_dict = word_max_usage_limit(df_word,"word","frequency", 3)
word_adjust_usage_dict

{'bir': 829,
 'bu': 487,
 'ne': 353,
 've': 342,
 'için': 241,
 'mi': 236,
 'o': 220,
 'ben': 216,
 'de': 215,
 'çok': 213,
 'ama': 205,
 'var': 193,
 'evet': 190,
 'mı': 176,
 'değil': 171,
 'da': 159,
 'şey': 158,
 'hayır': 150,
 'daha': 146,
 'sen': 144,
 'kadar': 118,
 'bana': 117,
 'yok': 109,
 'onu': 109,
 'seni': 108,
 'beni': 107,
 'bunu': 107,
 'gibi': 106,
 'iyi': 104,
 'tamam': 102,
 'her': 99,
 'benim': 98,
 'sana': 90,
 'ki': 85,
 'neden': 82,
 'zaman': 79,
 'senin': 79,
 'sadece': 79,
 'burada': 78,
 'nasıl': 72,
 'olduğunu': 71,
 'hiç': 69,
 'sonra': 68,
 'şimdi': 67,
 'en': 65,
 'ya': 64,
 'öyle': 63,
 'mu': 62,
 'şu': 60,
 'misin': 58,
 'hadi': 57,
 'önce': 56,
 'biraz': 55,
 'musun': 53,
 'güzel': 53,
 'oldu': 50,
 'böyle': 50,
 'ona': 49,
 'lütfen': 48,
 'bile': 46,
 'yani': 46,
 'artık': 45,
 'onun': 45,
 'geri': 44,
 'bak': 44,
 'istiyorum': 43,
 'peki': 42,
 'eğer': 42,
 'çünkü': 42,
 'biliyorum': 41,
 'gerçekten': 40,
 'başka': 40,
 'tek': 39,
 'olarak': 39,
 'be

In [42]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_adjust_usage_dict)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,sana gerek yok burada kal,ztZMp7dpRto
1,gerek yok çok teşekkür ederim ben zaten birkaç...,zpfhmsZB26w
2,pardon ben çok sıcak bir kahve alabilir miyim ...,zoN3xOGVjbk
3,bilir ne güzel anne olacaksın çok sevindim bil...,zmmMNX_dDg4
4,burası biraz büyük değil mi bize,zmmMNX_dDg4
...,...,...
1363,bana ne kadar güzel bir kadın olduğumu söyledi...,0EmRyNfzWKM
1364,ama küçük bir sorun var,0EmRyNfzWKM
1365,peki birisi için mi yaptın yoksa oyun olsun di...,0AOYUEuSiyw
1366,ben dün gece için senden çok özür dilerim,071PtpSHv5c


In [43]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
1363,ne var ki daha önce,130,138,ne var ki daha önce,_0Ozm1E1xHE
1364,o mu bu mu var evet,53,55,o mu bu mu var evet,StouY7hsuSQ
1365,daha genç bir at al,2041,2043,daha genç bir at al,W263ryKMPH0
1366,cidden mi o mu evet,368,370,cidden mi o mu evet,FythRb2U8OU


In [44]:
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Temp_Orj.xlsx", index=False)

In [45]:
df_word_count = word_count_result(df_youtube_word_usage_merge, df_youtube_word_usage_merge.columns[0:1])
df_word_count

,word,word_count
0,bir,560
1,bu,247
2,ben,216
3,de,215
4,çok,213
...,...,...
944,neyi,1
945,bahsediyorsun,1
946,sonsuza,1
947,gerektiğini,1


In [46]:
word_set = set(word_list)
sent_result_word_set = set(df_word_count["word"])

In [47]:
word_set.difference(sent_result_word_set)

{'adını',
 'ajan',
 'altına',
 'arka',
 'aşkına',
 'bay',
 'bayan',
 'bayanlar',
 'bayım',
 'beri',
 'beyler',
 'birden',
 'burda',
 'böylece',
 'canına',
 'dek',
 'dolayı',
 'durun',
 'ele',
 'evine',
 'görev',
 'görünüşe',
 'güç',
 'hangi',
 'hayatını',
 'hepiniz',
 'içki',
 'kapa',
 'kaptan',
 'olası',
 'olmalı',
 'parti',
 'pekala',
 'sağol',
 'silah',
 'sona',
 'sıkı',
 'tut',
 'tür',
 'uyuşturucu',
 'verici',
 'yeterince',
 'yola',
 'zamandır',
 'zavallı',
 'çekil',
 'çekilin',
 'çeviri',
 'çoktan',
 'öyleyse',
 'üzere'}

#### Youtube Tools

In [55]:
df_youtube_word_usage_merge = pd.read_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Temp_Orj.xlsx")
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
1363,ne var ki daha önce,130,138,ne var ki daha önce,_0Ozm1E1xHE
1364,o mu bu mu var evet,53,55,o mu bu mu var evet,StouY7hsuSQ
1365,daha genç bir at al,2041,2043,daha genç bir at al,W263ryKMPH0
1366,cidden mi o mu evet,368,370,cidden mi o mu evet,FythRb2U8OU


In [56]:
df_youtube_word_usage_merge_reassign = df_youtube_word_usage_merge.copy() # important because when it doesn't copy, main df are also change.

In [57]:
df_youtube_word_usage_merge_reassign

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
1363,ne var ki daha önce,130,138,ne var ki daha önce,_0Ozm1E1xHE
1364,o mu bu mu var evet,53,55,o mu bu mu var evet,StouY7hsuSQ
1365,daha genç bir at al,2041,2043,daha genç bir at al,W263ryKMPH0
1366,cidden mi o mu evet,368,370,cidden mi o mu evet,FythRb2U8OU


In [58]:
df_youtube_word_usage_merge_reassign["start_time"] = 1
df_youtube_word_usage_merge_reassign["end_time"] = 2

In [59]:
#df_youtube_word_usage_merge_reassign["end_time"] = df_youtube_word_usage_merge_reassign["start_time"] + 1

In [60]:
out_path = "/home/kurubal/Downloads"

In [61]:
df_youtube_word_usage_merge_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Temp.xlsx", index=False)

In [62]:
# read file after youtube tool result
df_youtube_word_usage_merge_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Temp.xlsx")  # from youtube tool
df_youtube_word_usage_merge_video_list = list(set(df_youtube_word_usage_merge_youtube_tool["video_id"]))

In [63]:
df_youtube_word_usage_merge_new = df_youtube_word_usage_merge[df_youtube_word_usage_merge["video_id"].isin(df_youtube_word_usage_merge_video_list)]
df_youtube_word_usage_merge_new

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
1285,bir de bu var o da mı bana,6141,6144,bir de bu var o da mı bana,P_KbNZi5uUo
1331,siz de su alır mısınız,196,199,siz de su alır mısınız,MXciFLdLVSY
1332,misin bir sorun mu var,5062,5064,i yi misin bir sorun mu var,g6yvs1f4S1s
1338,son bir kez dinle beni,8118,8120,son bir kez dinle beni,PeQifrSZmBo


In [64]:
df_youtube_word_usage_merge_new["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge_new

<ipython-input-64-c1453041db40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_word_usage_merge_new["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"


,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...
...,...,...,...,...,...,...
1285,bir de bu var o da mı bana,6141,6144,bir de bu var o da mı bana,P_KbNZi5uUo,https://www.youtube.com/watch?v=P_KbNZi5uUo&t=...
1331,siz de su alır mısınız,196,199,siz de su alır mısınız,MXciFLdLVSY,https://www.youtube.com/watch?v=MXciFLdLVSY&t=...
1332,misin bir sorun mu var,5062,5064,i yi misin bir sorun mu var,g6yvs1f4S1s,https://www.youtube.com/watch?v=g6yvs1f4S1s&t=...
1338,son bir kez dinle beni,8118,8120,son bir kez dinle beni,PeQifrSZmBo,https://www.youtube.com/watch?v=PeQifrSZmBo&t=...


In [65]:
#df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Result.xlsx", index=False)

In [66]:
df_youtube_word_usage_merge_new_count = word_count_result(df_youtube_word_usage_merge_new, df_youtube_word_usage_merge_new.columns[0:1])
df_youtube_word_usage_merge_new_count

,word,word_count
0,bir,155
1,ben,80
2,çok,78
3,şey,73
4,bu,66
...,...,...
711,nasılsın,1
712,mısınız,1
713,mısın,1
714,mümkün,1


#### Copy Move And Delete

In [68]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}*")  # ???
output_file

['Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6_Not_Word_Usage.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6_Temp_Orj.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6_Result.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Manuel Select Result Word Count

In [ ]:
manuel_select_path = "/home/kurubal/Downloads"  # from youtube tool
manuel_select_file = f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Result_Manuel.xlsx"

In [ ]:
df_sentence_ratio_result = pd.read_csv(f"{path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_All.csv")  # ?? 
df_sentence_ratio_result

In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio = pd.read_csv(f"{path}/\
{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}_Not_Word_Usage.csv")
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio.drop(["include_string_ratio"], axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

In [ ]:
df_word_group_time_loc_in_sent_result_repeat_ratio["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result_repeat_ratio['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result_repeat_ratio['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result_repeat_ratio

In [ ]:
df_youtube_word_usage_merge_manuel = pd.read_excel(f"{manuel_select_path}/{manuel_select_file}")
df_youtube_word_usage_merge_manuel

In [ ]:
df_word_count_manuel = word_count_result(df_youtube_word_usage_merge_manuel, df_youtube_word_usage_merge_manuel.columns[0:1])
df_word_count_manuel

In [ ]:
word_set = set(word_list)
select_sent_result_word_set = set(df_word_count_manuel["word"])
select_sent_result_word_id_set_list = list(set(df_youtube_word_usage_merge_manuel["video_id"]))

In [ ]:
word_set.difference(select_sent_result_word_set)

In [ ]:
remove_video_id_list = []
not_in_sent_word_list = list(word_set.difference(select_sent_result_word_set))
not_in_sent_word_list = not_in_sent_word_list + remove_video_id_list
not_in_sent_word_list 

In [ ]:
df_youtube_sent_result = df_word_group_time_loc_in_sent_result_repeat_ratio[~df_word_group_time_loc_in_sent_result_repeat_ratio["video_id"].isin(select_sent_result_word_id_set_list)]
df_youtube_sent_result

In [ ]:
df_not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_result, "search_string", 20000, simple=False)
#df_not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
df_not_in_sent_selected_word_list_result

In [ ]:
# for specific result
df_not_in_sent_selected_videoid_list = df_not_in_sent_selected_word_list_result.groupby(["search_string","start_time","end_time","sentence","video_id"])[["word"]].agg(lambda x: set(x))
df_not_in_sent_selected_videoid_list.reset_index(inplace=True)
df_not_in_sent_selected_videoid_list.sort_values(by="word", ascending=False, inplace=True, key=lambda x:x.str.len())
df_not_in_sent_selected_videoid_list.reset_index(drop=True, inplace=True)
df_not_in_sent_selected_videoid_list

In [ ]:
# the goal is to represent the most words with the fewest sentences 
select_sent_index_list = []
for index in range(len(df_not_in_sent_selected_videoid_list)):
    word_set = df_not_in_sent_selected_videoid_list.loc[index,"word"]
    if len(not_in_sent_word_list)>0:
        for word in word_set:
            if word in not_in_sent_word_list:
                not_in_sent_word_list.remove(word)
                select_sent_index_list.append(index)
            else:
                pass
    else:
        pass

select_set = set(select_sent_index_list)
df_result = pd.DataFrame()
for select_index in select_set:
    df_var = df_not_in_sent_selected_videoid_list.iloc[[select_index],]
    df_result = pd.concat([df_result, df_var], axis=0)
df_result

In [ ]:
not_in_sent_word_list  # no result found